<a href="https://colab.research.google.com/github/nkcong206/Travel-Recommendation-System/blob/main/database/database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [45]:
import psycopg2
import json
import getpass
import os

In [4]:
postgres_url = getpass.getpass("Enter your postgresql url: ")

Enter your postgresql url: ··········


In [30]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
# Connect to PostgreSQL database
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()
cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()

cur.execute("""
DROP SCHEMA IF EXISTS travel_database CASCADE;
CREATE SCHEMA IF NOT EXISTS travel_database;
SET search_path TO travel_database, public;  -- Use public where PostGIS is installed

-- Create a custom type for the Address structure
CREATE TYPE Address AS (
    street TEXT,
    district TEXT,
    city TEXT
);

CREATE TYPE Time AS (
    open_time TIME WITH TIME ZONE,
    close_time TIME WITH TIME ZONE
)

-- Create the Hotel table with an embedding vector column
CREATE TABLE Hotel (
    hotel_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address Address,
    location GEOMETRY(POINT),
    amenities TEXT[ ],
    styles TEXT[ ],
    rating DECIMAL(2, 1),
    description TEXT,
    img_url JSON,
    comments JSON
);

-- Create an index on the district of the Address and Rating columns for the Hotel table
-- CREATE INDEX idx_hotel_address_rating ON Hotel(((address).district), rating);

-- Create the Price table for the Hotel table
CREATE TABLE HotelPrice (
    hprice_id SERIAL PRIMARY KEY,
    hotel_id INT REFERENCES Hotel(hotel_id) ON DELETE CASCADE,
    room_type VARCHAR(255),
    capacity INT,
    price INT NOT NULL
);

-- Create an index on the hotel_id for the HotelPrice table
-- CREATE INDEX hotel_idx_address_district ON Hotel(((address).district));
-- CREATE INDEX hotel_idx_rating ON Hotel(rating);
-- CREATE INDEX hotel_idx_location ON Hotel USING GIST(location);

-- Create the TouristAttraction table with an embedding vector column
CREATE TABLE TouristAttraction (
    attraction_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address Address,
    location GEOMETRY(POINT),
    attraction_type VARCHAR(255),
    working_hour Time,
    rating DECIMAL(2, 1),
    tour_duration INTERVAL,
    description TEXT,
    img_url JSON,
    comments JSON
);

-- Create an index on the district of the Address and Rating columns for the TouristAttraction table
-- CREATE INDEX attraction_idx_address_district ON TouristAttraction(((address).district));
-- CREATE INDEX attraction_idx_rating ON TouristAttraction(rating);
-- CREATE INDEX attraction_idx_location ON TouristAttraction USING GIST(location);

-- Create the Price table for the TouristAttraction table
CREATE TABLE AttractionPrice (
    aprice_id SERIAL PRIMARY KEY,
    attraction_id INT REFERENCES TouristAttraction(attraction_id) ON DELETE CASCADE,
    ticket_type VARCHAR(255),
    number_people INT,
    price INT NOT NULL
);

-- Create an index on the attraction_id for the AttractionPrice table
-- CREATE INDEX idx_attractionprice_attractionid ON AttractionPrice(attraction_id);

-- Create the Restaurant table with an embedding vector column
CREATE TABLE Restaurant (
    res_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address Address,
    location GEOMETRY(POINT),
    working_hour Time,
    suitable_for TEXT[],
    restaurant_type TEXT[],
    rating DECIMAL(2, 1),
    description TEXT,
    parking_available BOOLEAN NOT NULL,
    kids_play_area BOOLEAN NOT NULL,
    img_url JSON,
    comments JSON
);

-- Create an index on the district of the Address and Rating columns for the Restaurant table
-- CREATE INDEX res_idx_address_district ON Restaurant(((address).district));
-- CREATE INDEX res_idx_rating ON Restaurant(rating);
-- CREATE INDEX res_idx_location ON Restaurant USING GIST(location);
""")
conn.commit()
cur.close()
conn.close()

Luxury,
Eco-friendly,
Boutique,
Beachfront,
Family-friendly,
Business,
Romantic,
Cultural,
Wellness,
Adventure


In [50]:
import base64, httpx
from langchain_core.messages import HumanMessage
data = {
  "img_url": [
    "https://ik.imagekit.io/tvlk/apr-asset/Ixf4aptF5N2Qdfmh4fGGYhTN274kJXuNMkUAzpL5HuD9jzSxIGG5kZNhhHY-p7nw/hotel/asset/20075775-804e265fb0bc601cf1757992608dc0c8.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-332,pr-true,q-80,w-480",
    "https://ik.imagekit.io/tvlk/apr-asset/Ixf4aptF5N2Qdfmh4fGGYhTN274kJXuNMkUAzpL5HuD9jzSxIGG5kZNhhHY-p7nw/hotel/asset/20075775-24f1983ad091e2f3ec5587076cb5aa0d.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/Ixf4aptF5N2Qdfmh4fGGYhTN274kJXuNMkUAzpL5HuD9jzSxIGG5kZNhhHY-p7nw/hotel/asset/20075775-443207d9a60d8b8ac0fc4eb4cddb5a8b.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/Ixf4aptF5N2Qdfmh4fGGYhTN274kJXuNMkUAzpL5HuD9jzSxIGG5kZNhhHY-p7nw/hotel/asset/20075775-05dc1d66750934106c299c5499b2ebcb.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/Ixf4aptF5N2Qdfmh4fGGYhTN274kJXuNMkUAzpL5HuD9jzSxIGG5kZNhhHY-p7nw/hotel/asset/20075775-9d3fcde1dcf5dcde5df95332394daa8c.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/Ixf4aptF5N2Qdfmh4fGGYhTN274kJXuNMkUAzpL5HuD9jzSxIGG5kZNhhHY-p7nw/hotel/asset/20075775-9ab065ca77d5217aad30ef5ac322906b.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184",
    "https://ik.imagekit.io/tvlk/apr-asset/Ixf4aptF5N2Qdfmh4fGGYhTN274kJXuNMkUAzpL5HuD9jzSxIGG5kZNhhHY-p7nw/hotel/asset/20075775-8158b571c9cd086c6faeb80865ae15b1.jpeg?_src=imagekit&tr=c-at_max,f-jpg,fo-auto,h-162,pr-true,q-80,w-184"
  ]
}
image_url = data["img_url"][:4]
image_data = base64.b64encode(httpx.get(image_url[0]).content).decode("utf-8")

prompt = "Identify the hotel theme from the images. Return only one theme, and it must be one of the following: 'Luxury', 'Eco-friendly', 'Boutique', 'Beachfront', 'Family-friendly', 'Business', 'Romantic', 'Cultural', 'Wellness', or 'Adventure'. Do not provide any other information."

content=[
        {"type": "text", "text": prompt},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]

message = HumanMessage(
    content=content
)

In [51]:
respone = llm.invoke([message])
print(respone.content)

Luxury



In [ ]:
# Dữ liệu mẫu
hotels = [
    ("Khách sạn A", "Khách sạn cao cấp gần trung tâm Hà Nội, với các tiện nghi hiện đại và dịch vụ khách hàng xuất sắc.", 4.5)
]

attractions = [
    ("Tháp Rùa", "Một biểu tượng nổi tiếng của Hà Nội, nằm trên hồ Hoàn Kiếm, rất phổ biến với du khách và người dân địa phương.", "Di tích lịch sử và điểm tham quan nổi bật", 4.7),
    ("Chùa Một Cột", "Chùa cổ kính với kiến trúc độc đáo, nổi tiếng với hình dáng như một bông sen và nằm trên một cột duy nhất.", "Di tích văn hóa và tâm linh", 4.6),
    ("Lăng Bác", "Lăng chủ tịch Hồ Chí Minh, nơi lưu giữ di hài của Hồ Chí Minh, một điểm đến quan trọng và thiêng liêng ở Hà Nội.", "Nơi thờ cúng và lưu giữ di sản", 4.8),
    ("Văn Miếu Quốc Tử Giám", "Trường đại học cổ xưa nhất của Việt Nam, nơi thờ các vị hiền triết và là một biểu tượng của giáo dục truyền thống.", "Di tích giáo dục và văn hóa", 4.9),
    ("Hồ Tây", "Hồ lớn nhất tại Hà Nội, là nơi lý tưởng cho các hoạt động giải trí, đi dạo và thư giãn bên bờ hồ xanh mát.", "Khu vực giải trí và thư giãn", 4.4)
]

restaurants = [
    ("Nhà hàng A", "Nhà hàng phục vụ các món ăn địa phương với hương vị đậm đà, nổi tiếng với món bún chả và phở.", 4.2),
    ("Nhà hàng B", "Nhà hàng phục vụ món Âu và Á, với thực đơn đa dạng và phong cách ẩm thực quốc tế.", 3.9),
    ("Nhà hàng C", "Nhà hàng hải sản tươi ngon, chuyên cung cấp các món hải sản tươi sống và món đặc sản vùng biển.", 4.3),
    ("Nhà hàng D", "Nhà hàng sang trọng với không gian ấm cúng, phù hợp cho các bữa tối lãng mạn và sự kiện đặc biệt.", 4.6),
    ("Nhà hàng E", "Nhà hàng nhanh và tiện lợi, phục vụ các món ăn nhanh và đồ uống, lý tưởng cho bữa trưa gọn gàng.", 4.0)
]


# Kết nối đến cơ sở dữ liệu PostgreSQL
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()
cur.execute("""SET search_path TO travel_database, public;""")

# Chèn dữ liệu vào bảng Hotel
for name, description, rating in hotels:
    cur.execute("""
        INSERT INTO Hotel (name, address, location, rating, description, img_url, comments)
        VALUES (%s, ROW('0', 'Cầu Giấy', 'Hà Nội'), ST_Point(105.8498, 21.0285), %s, %s, %s, %s);
    """, (name, rating, description, json.dumps({"img": "url"}), json.dumps({"comments": "none"})))

# Chèn dữ liệu vào bảng TouristAttraction
for name, description, attraction_type, rating in attractions:
    cur.execute("""
        INSERT INTO TouristAttraction (name, address, location, attraction_type, rating, tour_duration, description, img_url, comments)
        VALUES (%s, ROW('0', 'Cầu Giấy', 'Hà Nội'), ST_Point(105.8498, 21.0285), %s, %s, NULL, %s, %s, %s);
    """, (name, attraction_type, rating, json.dumps({"img": "url"}), json.dumps({"comments": "none"})))

# Chèn dữ liệu vào bảng Restaurant
for name, description, rating in restaurants:
    cur.execute("""
        INSERT INTO Restaurant (name, address, location, rating, description, img_url, comments)
        VALUES (%s, ROW('0', 'Cầu Giấy', 'Hà Nội'), ST_Point(105.8498, 21.0285), %s, %s, %s, %s);
    """, (name, rating, description, json.dumps({"img": "url"}), json.dumps({"comments": "none"})))

# Commit các thay đổi
conn.commit()

# Đóng kết nối và con trỏ
cur.close()
conn.close()